In [1]:
!pip install geoopt
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [3]:
from huggingface_hub import login

#Enter your token here
HF_TOKEN_NAMAN = ''
login(HF_TOKEN_NAMAN)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

llama_path = 'meta-llama/Llama-2-7b-hf'
llama_model = AutoModelForCausalLM.from_pretrained(llama_path, trust_remote_code=True,
                                                   output_hidden_states=True,
                                                  )
llama_model.to(device)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [6]:
tokenizer = AutoTokenizer.from_pretrained(llama_path, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("wikipedia", "20220301.simple", split="train[:100]", trust_remote_code=True)
texts = dataset["text"]

In [ ]:
def process_text_in_chunks(text, tokenizer, model, chunk_size=512, max_length=512):
    """
    Process a single text input in smaller chunks to avoid memory overflow.
    Returns accumulated activations for each layer.
    """
    # Tokenize text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    input_ids = tokens["input_ids"][0]  # Get the first sequence

    # Get the number of layers from the model's configuration
    num_layers = model.config.num_hidden_layers

    # Initialize accumulation list on CPU
    accumulated_hidden_states = [torch.zeros(0, model.config.hidden_size).to("cpu") for _ in range(num_layers)]

    # Process input in chunks
    for i in range(0, len(input_ids), chunk_size):
        chunk = input_ids[i : i + chunk_size].unsqueeze(0).to("cuda")  # Move to GPU

        with torch.no_grad():
            outputs = model(input_ids=chunk, output_hidden_states=True)
            hidden_states = outputs.hidden_states  # List of tensors, one per layer
            #print("Length of hidden states", len(hidden_states))
        # Offload activations for each layer to CPU and accumulate
        for layer_idx, layer_activation in enumerate(hidden_states[1:]):
            #print(layer_idx)
            accumulated_hidden_states[layer_idx] = torch.cat(
                [accumulated_hidden_states[layer_idx], layer_activation.squeeze(0).to("cpu")], dim=0
            )

        # Free GPU memory
        del chunk, hidden_states
        torch.cuda.empty_cache()

    return accumulated_hidden_states

# Step 4: Process all samples efficiently
def process_all_samples(texts, tokenizer, model, chunk_size=512):
    """
    Process all texts and compute accumulated hidden states for each layer.
    """
    # Get the number of layers from the model's configuration
    num_layers = model.config.num_hidden_layers

    # Initialize global accumulation list for all layers
    global_accumulated_hidden_states = [torch.zeros(0, model.config.hidden_size).to("cpu") for _ in range(num_layers)]

    for text in texts:
        sample_hidden_states = process_text_in_chunks(text, tokenizer, model, chunk_size)

        # Accumulate results across all samples for each layer
        for layer_idx, layer_activation in enumerate(sample_hidden_states):
            global_accumulated_hidden_states[layer_idx] = torch.cat(
                [global_accumulated_hidden_states[layer_idx], layer_activation], dim=0
            )
        # Free CPU memory for intermediate sample_hidden_states
        del sample_hidden_states

    return global_accumulated_hidden_states


def compute_layer_averages(global_accumulated_hidden_states, num_samples):
    """
    Compute average hidden states for each layer, collapsing both tokens and samples.
    """
    return [layer_hidden_states.mean(dim=0, keepdim=True).mean(dim=0, keepdim=True) for layer_hidden_states in global_accumulated_hidden_states]

# Execute the pipeline
chunk_size = 128
global_hidden_states = process_all_samples(texts, tokenizer, llama_model, chunk_size)
averaged_hidden_states = compute_layer_averages(global_hidden_states, len(texts))

In [ ]:
# import pickle
# with open('averaged_hidden_states.pkl', 'wb') as f:
#     pickle.dump(averaged_hidden_states, f)

In [7]:
import pickle
with open('averaged_hidden_states.pkl', 'rb') as f:
    averaged_hidden_states = pickle.load(f)

In [8]:
import math

import torch
from torch import Tensor

def exp_map0(x: Tensor, curv: float | Tensor = 1.0, eps: float = 1e-8) -> Tensor:
    """
    Map points from the tangent space at the vertex of hyperboloid, on to the
    hyperboloid. This mapping is done using the exponential map of Lorentz model.

    Args:
        x: Tensor of shape `(B, D)` giving batch of Euclidean vectors to project
            onto the hyperboloid. These vectors are interpreted as velocity
            vectors in the tangent space at the hyperboloid vertex.
        curv: Positive scalar denoting negative hyperboloid curvature.
        eps: Small float number to avoid division by zero.

    Returns:
        Tensor of same shape as `x`, giving space components of the mapped
        vectors on the hyperboloid.
    """

    rc_xnorm = curv**0.5 * torch.norm(x, dim=-1, keepdim=True)

    # Ensure numerical stability in sinh by clamping input.
    sinh_input = torch.clamp(rc_xnorm, min=eps, max=math.asinh(2**15))
    _output = torch.sinh(sinh_input) * x / torch.clamp(rc_xnorm, min=eps)
    return _output

def pairwise_inner(x: Tensor, y: Tensor, curv: float | Tensor = 1.0):
    """
    Compute pairwise Lorentzian inner product between input vectors.

    Args:
        x: Tensor of shape `(B1, D)` giving a space components of a batch
            of vectors on the hyperboloid.
        y: Tensor of shape `(B2, D)` giving a space components of another
            batch of points on the hyperboloid.
        curv: Positive scalar denoting negative hyperboloid curvature.
        eps: Small float number to avoid numerical instability.

    Returns:
        Tensor of shape `(B1, B2)` giving pairwise Lorentzian inner product
        between input vectors.
    """

    x_time = torch.sqrt(1 / curv + torch.sum(x**2, dim=-1, keepdim=True))
    y_time = torch.sqrt(1 / curv + torch.sum(y**2, dim=-1, keepdim=True))
    xyl = x @ y.T - x_time @ y_time.T
    return xyl


def pairwise_dist(
    x: Tensor, y: Tensor, curv: float | Tensor = 1.0, eps: float = 1e-8
) -> Tensor:
    """
    Compute the pairwise geodesic distance between two batches of points on
    the hyperboloid.

    Args:
        x: Tensor of shape `(B1, D)` giving a space components of a batch
            of point on the hyperboloid.
        y: Tensor of shape `(B2, D)` giving a space components of another
            batch of points on the hyperboloid.
        curv: Positive scalar denoting negative hyperboloid curvature.
        eps: Small float number to avoid numerical instability.

    Returns:
        Tensor of shape `(B1, B2)` giving pairwise distance along the geodesics
        connecting the input points.
    """

    c_xyl = -curv * pairwise_inner(x, y, curv)
    _distance = torch.acosh(torch.clamp(c_xyl, min=1 + eps))
    return _distance / curv**0.5



In [9]:
import torch

# Step 1: Project each layer's hidden state onto the hyperboloid
def project_layers_to_hyperboloid(averaged_hidden_states, curv=1.0):
    """
    Projects each layer's averaged hidden state onto the Lorentz hyperboloid.
    Args:
        averaged_hidden_states (list of torch.Tensor): List of tensors, each of shape [1, hidden_size].
        curv (float): Curvature of the hyperboloid.
    Returns:
        torch.Tensor: Projected embeddings on the hyperboloid of shape [num_layers, hidden_size].
    """
    # Apply exp_map0 to each tensor in the list
    projected_embeddings = torch.cat([exp_map0(state, curv=curv) for state in averaged_hidden_states], dim=0)
    return projected_embeddings

proj_embeddings = project_layers_to_hyperboloid(averaged_hidden_states, curv=1.0)

print("proj_embeddings:")
print(proj_embeddings)


proj_embeddings:
tensor([[-2.7793e-04,  8.2473e-04, -1.7620e-03,  ...,  3.0238e-03,
          3.9035e-04,  3.8933e-03],
        [ 1.5083e-01,  4.0349e+00, -5.2132e+00,  ...,  8.6724e+00,
          1.0077e+01, -1.1917e+00],
        [-7.5827e-01,  6.7320e+00, -6.3043e+00,  ...,  9.2440e+00,
          1.2436e+01, -3.1147e+00],
        ...,
        [ 5.3497e+01, -2.3036e+02,  7.6928e+01,  ..., -1.9112e+02,
         -2.5501e+01, -3.2021e+01],
        [ 1.7715e+02, -3.8207e+02,  3.6704e+02,  ..., -2.8224e+02,
          1.7800e+02,  8.7722e+01],
        [ 3.5627e+02, -8.6981e+02,  5.1998e+02,  ..., -4.0164e+02,
          2.1051e+02, -2.9130e+02]])


In [10]:
proj_embeddings.shape

torch.Size([32, 4096])

In [12]:
import numpy as np
import torch

def find_top_k_pairs(affinity_matrix, k=5):
    """
    Find top k pairs of layers based on similarity, ensuring each layer appears only once.

    Parameters:
    - affinity_matrix: Similarity matrix between layers
    - k: Number of top pairs to return

    Returns:
    - top_pairs: List of tuples, each containing two layer indices and their similarity
    """
    # Convert to numpy if it's a torch tensor
    if torch.is_tensor(affinity_matrix):
        affinity_matrix = affinity_matrix.numpy()

    # Get the total number of layers
    n_layers = affinity_matrix.shape[0]

    # Create a mask to prevent reusing layers
    used_layers = set()
    top_pairs = []

    # Flatten the upper triangle of the affinity matrix (excluding diagonal)
    pairs = []
    for i in range(n_layers):
        for j in range(i+1, n_layers):
            pairs.append((i, j, affinity_matrix[i, j]))

    # Sort pairs by similarity in descending order
    pairs.sort(key=lambda x: x[2], reverse=True)

    # Select top k pairs, ensuring no layer is repeated
    for pair in pairs:
        if len(top_pairs) == k:
            break

        i, j, similarity = pair

        # Check if either layer has been used
        if i not in used_layers and j not in used_layers:
            top_pairs.append((i, j, similarity))
            used_layers.add(i)
            used_layers.add(j)

    return top_pairs

def hyperbolic_inner_product(x, y, curv=1.0):
    x_time = torch.sqrt(1 / curv + torch.sum(x**2, dim=-1, keepdim=True))
    y_time = torch.sqrt(1 / curv + torch.sum(y**2, dim=-1, keepdim=True))
    return x @ y.T - x_time @ y_time.T

flat_embeddings = proj_embeddings.squeeze(1)
affinity_matrix = hyperbolic_inner_product(flat_embeddings, flat_embeddings).cpu().numpy()

affinity_matrix = (affinity_matrix - affinity_matrix.min()) / (affinity_matrix.max() - affinity_matrix.min())

top_pairs = find_top_k_pairs(affinity_matrix, k=10)

print("Top Pairs of Layers:")
for i, (layer1, layer2, similarity) in enumerate(top_pairs, 1):
    print(f"Pair {i}: Layers {layer1} and {layer2}, Similarity = {similarity:.4f}")

Top Pairs of Layers:
Pair 1: Layers 0 and 30, Similarity = 1.0000
Pair 2: Layers 1 and 2, Similarity = 0.9999
Pair 3: Layers 4 and 5, Similarity = 0.9997
Pair 4: Layers 7 and 8, Similarity = 0.9995
Pair 5: Layers 9 and 10, Similarity = 0.9992
Pair 6: Layers 11 and 12, Similarity = 0.9990
Pair 7: Layers 13 and 14, Similarity = 0.9989
Pair 8: Layers 3 and 6, Similarity = 0.9984
Pair 9: Layers 22 and 23, Similarity = 0.9978
Pair 10: Layers 16 and 17, Similarity = 0.9976


In [16]:
def merge_layer_pairs(model, layer_pairs):
    """
    Merge specified pairs of layers by averaging their weights.

    Parameters:
    - model: The neural network model
    - layer_pairs: List of pairs to merge, can be either:
      1. List of (layer1, layer2) tuples
      2. List of (layer1, layer2, similarity) triples

    Returns:
    - model: The modified model with layers merged
    """
    # Extract just the layer indices if triples are provided
    if layer_pairs and len(layer_pairs[0]) == 3:
        sorted_pairs = sorted([(pair[0], pair[1]) for pair in layer_pairs],
                               key=lambda x: max(x),
                               reverse=True)
    else:
        sorted_pairs = sorted(layer_pairs, key=lambda x: max(x), reverse=True)

    for base_layer_idx, merge_layer_idx in sorted_pairs:
        print(f"Merging layer {merge_layer_idx} into layer {base_layer_idx}")

        base_layer = model.model.layers[base_layer_idx]
        merging_layer = model.model.layers[merge_layer_idx]

        # MLP layers
        # gate_proj: Average weights
        base_layer.mlp.gate_proj.weight.data = (
            base_layer.mlp.gate_proj.weight.data + merging_layer.mlp.gate_proj.weight.data
        ) / 2

        # down_proj: Average weights
        base_layer.mlp.down_proj.weight.data = (
            base_layer.mlp.down_proj.weight.data + merging_layer.mlp.down_proj.weight.data
        ) / 2

        # up_proj: Average weights
        base_layer.mlp.up_proj.weight.data = (
            base_layer.mlp.up_proj.weight.data + merging_layer.mlp.up_proj.weight.data
        ) / 2

        # Self-Attention layers
        # q_proj: Average weights
        base_layer.self_attn.q_proj.weight.data = (
            base_layer.self_attn.q_proj.weight.data + merging_layer.self_attn.q_proj.weight.data
        ) / 2

        # k_proj: Average weights
        base_layer.self_attn.k_proj.weight.data = (
            base_layer.self_attn.k_proj.weight.data + merging_layer.self_attn.k_proj.weight.data
        ) / 2

        # v_proj: Average weights
        base_layer.self_attn.v_proj.weight.data = (
            base_layer.self_attn.v_proj.weight.data + merging_layer.self_attn.v_proj.weight.data
        ) / 2

        # o_proj: Average weights
        base_layer.self_attn.o_proj.weight.data = (
            base_layer.self_attn.o_proj.weight.data + merging_layer.self_attn.o_proj.weight.data
        ) / 2

        # Remove the merged layer
        print(f"Deleting layer {merge_layer_idx}")
        del model.model.layers[merge_layer_idx]

    return model

In [17]:
# Merge clusters
merged_model = merge_layer_pairs(llama_model, top_pairs)


Merging layer 30 into layer 0
Deleting layer 30
Merging layer 23 into layer 22
Deleting layer 23
Merging layer 17 into layer 16
Deleting layer 17
Merging layer 14 into layer 13
Deleting layer 14
Merging layer 12 into layer 11
Deleting layer 12
Merging layer 10 into layer 9
Deleting layer 10
Merging layer 8 into layer 7
Deleting layer 8
Merging layer 6 into layer 3
Deleting layer 6
Merging layer 5 into layer 4
Deleting layer 5
Merging layer 2 into layer 1
Deleting layer 2


In [18]:
merged_model.config.num_hidden_layers = len(merged_model.model.layers)
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text(model, tokenizer, prompt, max_length=64, num_return_sequences=1):
    """
    Generates text from the given model and prompt.
    Args:
        model: HuggingFace transformer model (e.g., LLaMA).
        tokenizer: Corresponding tokenizer for the model.
        prompt (str): Input text prompt for generation.
        max_length (int): Maximum length of generated text.
        num_return_sequences (int): Number of text sequences to generate.
    Returns:
        List[str]: List of generated text sequences.
    """
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate text
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,  # Prevent repetitive text
        early_stopping=True
    )

    # Decode and return the generated sequences
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


# # Generate text with the unpruned model
prompt = "Today is a beautiful day"
# unpruned_text = generate_text(llama_model, tokenizer, prompt)
# print("Unpruned Model Output:")
# print(unpruned_text)

# Generate text with the pruned model
pruned_text = generate_text(merged_model, tokenizer, prompt)
print("\nPruned Model Output:")
print(pruned_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



Pruned Model Output:
['Today is a beautiful dayOO\nOCO\x00OJO1O?O OOWOAOUO-OIO0O9O2OROKO OEOLOHOOsOGOTO??OBOVONO']


In [30]:
top_pairs

[(0, 30, 0.99997246),
 (1, 2, 0.9999014),
 (4, 5, 0.999725),
 (7, 8, 0.9995125),
 (9, 10, 0.9991696),
 (11, 12, 0.9989782),
 (13, 14, 0.9988692),
 (3, 6, 0.9983898),
 (22, 23, 0.9978273),
 (16, 17, 0.9975614)]

In [20]:
dataset = load_dataset("wikipedia", "20220301.simple", split="train[:30]", trust_remote_code=True)
sentences = dataset["text"]

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [28]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer

def calculate_perplexity(merged_model, sentences, tokenizer=None, device='cuda'):
    """
    Calculate perplexity for a list of sentences using the merged model.

    Parameters:
    - merged_model: The merged language model
    - sentences: List of input sentences to calculate perplexity for
    - tokenizer: Tokenizer for the model (will use the model's default if not provided)
    - device: Device to run calculations on (default: 'cuda')

    Returns:
    - Perplexity score (lower is better)
    """
    if tokenizer is None:
        try:
            tokenizer = AutoTokenizer.from_pretrained(merged_model.config._name_or_path)
        except Exception as e:
            raise ValueError("No tokenizer provided and could not load default tokenizer") from e

    merged_model = merged_model.to(device)
    merged_model.eval()

    total_negative_log_likelihood = 0
    total_tokens = 0

    with torch.no_grad():
        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt",
                               add_special_tokens=True).to(device)

            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']

            outputs = merged_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            logits = outputs.logits
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()

            log_probs = F.log_softmax(shift_logits, dim=-1)

            selected_log_probs = log_probs.gather(-1, shift_labels.unsqueeze(-1)).squeeze(-1)

            mask = (shift_labels != tokenizer.pad_token_id).float()
            selected_log_probs *= mask

            total_negative_log_likelihood -= selected_log_probs.sum().item()
            total_tokens += mask.sum().item()

    if total_tokens == 0:
        raise ValueError("No valid tokens found in the input sentences")

    perplexity = math.exp(total_negative_log_likelihood / total_tokens)

    return perplexity

In [29]:
calculate_perplexity(merged_model, sentences, tokenizer)

46.32139210865856